In [47]:
import pandas as pd
import requests
from config import api_key
import json

In [49]:
dem_candidate=pd.read_csv("dem_candidate.csv")
dem_finance=pd.read_csv("new_dem_finance.csv")

In [50]:
dem_candidate=dem_candidate.rename(columns={
    "state":"candidate_state"
})
dem_finance=dem_finance.rename(columns={
    "state":"donor_state"
})

In [51]:
dem_data=pd.DataFrame.merge(dem_candidate, dem_finance, on="candidate_id")

In [52]:
dem_data.head(1)

,Unnamed: 0_x,name,candidate_id,candidate_state,party,incumbent_challenge_full,office_full,cycles,election_years,party.1,candidate_status,Unnamed: 0_y,donor_state,donation_count,total_raised,cycle
0,0,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],DEM,P,0,AZ,114,21611.44,2018


In [53]:
# create data frame with totals fundraised by every candidate in every state they fundraised

dem_data=dem_data.filter(["name","candidate_id","candidate_state","party","incumbent_challenge_full","office_full","cycles","election_years",
                        "candidate_status","donor_state","donation_count","total_raised"])
dem_data

,name,candidate_id,candidate_state,party,incumbent_challenge_full,office_full,cycles,election_years,candidate_status,donor_state,donation_count,total_raised
0,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,AZ,114,21611.44
1,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,CA,5,600.00
2,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,IL,8,1800.85
3,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,MA,3,1500.00
4,"ABBOUD, DEEDRA",S8AZ00122,AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,MI,1,250.00
...,...,...,...,...,...,...,...,...,...,...,...,...
2152,"WILSON, JENNY",S8UT00143,UT,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,UT,2557,806854.96
2153,"WILSON, JENNY",S8UT00143,UT,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,VA,3,4737.00
2154,"WILSON, JENNY",S8UT00143,UT,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,WA,7,4270.00
2155,"WILSON, JENNY",S8UT00143,UT,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,WV,2,1000.00


In [54]:
# if candidate_state = donor_state, in_state=True, if not, then False

in_state=(dem_data.candidate_state==dem_data.donor_state)
in_state=in_state.to_frame()
in_state=in_state.rename(columns={
    0: "in_state"
})
in_state.head()

,in_state
0,True
1,False
2,False
3,False
4,False


In [55]:
dem_data=pd.DataFrame.merge(dem_data,in_state, left_index=True, right_index=True)

dem_data.set_index("candidate_id")

,name,candidate_state,party,incumbent_challenge_full,office_full,cycles,election_years,candidate_status,donor_state,donation_count,total_raised,in_state
candidate_id,,,,,,,,,,,,
S8AZ00122,"ABBOUD, DEEDRA",AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,AZ,114,21611.44,True
S8AZ00122,"ABBOUD, DEEDRA",AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,CA,5,600.00,False
S8AZ00122,"ABBOUD, DEEDRA",AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,IL,8,1800.85,False
S8AZ00122,"ABBOUD, DEEDRA",AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,MA,3,1500.00,False
S8AZ00122,"ABBOUD, DEEDRA",AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,MI,1,250.00,False
...,...,...,...,...,...,...,...,...,...,...,...,...
S8UT00143,"WILSON, JENNY",UT,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,UT,2557,806854.96,True
S8UT00143,"WILSON, JENNY",UT,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,VA,3,4737.00,False
S8UT00143,"WILSON, JENNY",UT,DEM,Open seat,Senate,"[2018, 2020]",[2018],P,WA,7,4270.00,False


In [66]:
# for every candidate, add up donor_count, and total_raised, if in_state == False in order to have totals for out of state and in state

aggregate=dem_data.groupby(["candidate_id", "in_state"]).agg({"donation_count":["sum"], "total_raised":["sum"]})



aggregate.columns = aggregate.columns.droplevel(1)

aggregate=aggregate.reset_index()

in_state=aggregate.loc[aggregate["in_state"]==True]
in_state
out_of_state=aggregate.loc[aggregate["in_state"]==False]
out_of_state

state_agg=pd.DataFrame.merge(in_state, out_of_state, on="candidate_id", suffixes=("_in_state","_out_of_state"), how="outer")
state_agg.to_csv("dem_finance_data.csv")
state_agg
# i want this to have one row per candidate_id, with columns for in_state count,in_state_sum, as well as out_of_state count and sum

,candidate_id,in_state_in_state,donation_count_in_state,total_raised_in_state,in_state_out_of_state,donation_count_out_of_state,total_raised_out_of_state
0,H2MA04073,True,2773.0,907995.49,False,5166.0,1833851.36
1,S0AL00156,True,6465.0,1974748.72,False,72744.0,9000160.89
2,S0CA00199,True,7083.0,4228502.61,False,1997.0,1478827.85
3,S0NY00402,True,1.0,500.00,False,3.0,750.00
4,S0NY00410,True,21155.0,7975154.25,False,17145.0,6100790.98
...,...,...,...,...,...,...,...
64,S8WV00119,True,45.0,10869.00,False,301.0,26522.07
65,S8WY00189,True,775.0,316742.65,False,466.0,384726.92
66,S6NV00234,NaN,NaN,NaN,False,7.0,650.00
67,S8MD00260,NaN,NaN,NaN,False,110.0,80601.00


In [57]:
dem_candidate.reset_index("candidate_id")

,Unnamed: 0,name,candidate_state,party,incumbent_challenge_full,office_full,cycles,election_years,party.1,candidate_status
candidate_id,,,,,,,,,,
S8AZ00122,0,"ABBOUD, DEEDRA",AZ,DEM,Open seat,Senate,"[2018, 2020]",[2018],DEM,P
S8CT00105,1,"ADAMS, ANN MARIE DR",CT,DEM,Challenger,Senate,[2018],[2018],DEM,N
S2WI00219,2,"BALDWIN, TAMMY",WI,DEM,Incumbent,Senate,"[2012, 2014, 2016, 2018, 2020]","[2012, 2018, 2024]",DEM,F
S8MS00253,3,"BARIA, DAVID WAYNE",MS,DEM,Open seat,Senate,"[2018, 2020]",[2018],DEM,P
S8MS00295,4,"BARTEE, TOBEY",MS,DEM,Challenger,Senate,"[2018, 2020]","[2018, 2020]",DEM,N
...,...,...,...,...,...,...,...,...,...,...
S8WY00189,16,"TRAUNER, GARY",WY,DEM,Challenger,Senate,"[2018, 2020]",[2018],DEM,P
S8UT00150,17,"VICE, MITCHELL KENT",UT,DEM,Open seat,Senate,"[2018, 2020]",[2018],DEM,N
S2MA00170,18,"WARREN, ELIZABETH",MA,DEM,Incumbent,Senate,"[2012, 2014, 2016, 2018, 2020]","[2012, 2018, 2024]",DEM,F


In [172]:
# merge aggregate with dem_candidate to have all needed data (demographic data and total fundraising for in/out of state)
# one issue is several candidates are showing no $ for in_state, though out_of_state appears to be correct. I will need to pull a list of every candidate without in_state donations shown and then pull that data from a different end point or do it manually

In [67]:
final_dem_data=pd.DataFrame.merge(dem_candidate,state_agg, on="candidate_id")

In [70]:
final_dem_data=final_dem_data.filter(["name", "candidate_id", "candidate_state", "party", "incumbent_challenge_full", "cycles", "donation_count_in_state", "total_raised_in_state","donation_count_out_of_state", "total_raised_out_of_state"])

In [73]:
final_dem_data.to_csv("final_dem_data.csv")